In [3]:
"""
import pandas_datareader.data as web
import datetime

start = datetime.date(2018,1,1)
end = datetime.date.today()
data = web.DataReader('KO', 'yahoo', start, end)
今はyahoo使えない。yfinance使うとできる。
"""

"\nimport pandas_datareader.data as web\nimport datetime\n\nstart = datetime.date(2018,1,1)\nend = datetime.date.today()\ndata = web.DataReader('KO', 'yahoo', start, end)\n今はyahoo使えない。yfinance使うとできる。\n"

In [1]:
import yfinance as yf
from pandas_datareader import data as web
import datetime

#なんのデータを得るかを入力
tickers = ['AAPL']

#取得開始日を入力
start = "2022-01-02"

#取得終了日を入力
end = datetime.date.today()

#Yahoofinanceから取得するように設定
yf.pdr_override()

#データの取得を実行
data = web.get_data_yahoo(tickers, start, end)

[*********************100%%**********************]  1 of 1 completed


In [2]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,179.724548,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,177.443558,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,172.723602,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,169.840240,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,170.008102,86709100
...,...,...,...,...,...,...
2024-04-29,173.369995,176.029999,173.100006,173.500000,173.500000,68169400
2024-04-30,173.330002,174.990005,170.000000,170.330002,170.330002,65934800
2024-05-01,169.580002,172.710007,169.110001,169.300003,169.300003,50383100


In [5]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

#例として二つの単純移動平均線（SMA）のゴールデンクロスとデッドクロスで売買

class SmaCross(Strategy):
    n1 = 10 # 短期SMA
    n2 = 30 # 長期SMA

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1) 
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self): #チャートデータの行ごとに呼び出される
        if crossover(self.sma1, self.sma2): #sma1がsma2を上回った時
            self.buy() # 買い
        elif crossover(self.sma2, self.sma1):
            self.position.close() # 売り


In [7]:
bt = Backtest(
    data, # チャートデータ
    SmaCross, # 売買戦略
    cash=10000, # 最初の所持金(低いとエラーの可能性あり)
    commission=0.00005, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
bt.plot() # 実行結果（グラフ）、WSLでは表示されないっぽい


Start                     2022-01-03 00:00:00
End                       2023-11-28 00:00:00
Duration                    694 days 00:00:00
Exposure Time [%]                   50.521921
Equity Final [$]                 10830.102435
Equity Peak [$]                  12499.773829
Return [%]                           8.301024
Buy & Hold Return [%]                4.609637
Return (Ann.) [%]                     4.28457
Volatility (Ann.) [%]               18.458724
Sharpe Ratio                         0.232116
Sortino Ratio                        0.380036
Calmar Ratio                         0.199199
Max. Drawdown [%]                  -21.509008
Avg. Drawdown [%]                   -3.869822
Max. Drawdown Duration      286 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                    9
Win Rate [%]                        44.444444
Best Trade [%]                      17.518143
Worst Trade [%]                     -6.845818
Avg. Trade [%]                    

Row(id='1449', ...)

In [8]:
"""
Start	バックテスト開始日時
End	バックテスト終了日時
Duration	バックテスト期間
Exposure Time [%]	ポジションを保有していた期間の割合
Equity Final [$]	最終的な資金
Equity Peak [$]	資金の最大値
Return [%]	損益
Buy & Hold Retun [%]	バイ＆ホールドしていた場合の損益
Return (Ann.) [%]	年間損益
Volatility (Ann.） [%]	年間損益のボラティリティ
Sharp Ratio	シャープレシオ
リターンとリスク（標準偏差）の比から計算される
値が大きい方がいい
Sortino Ratio	ソルティノレシオ
下落時のリスクを考慮して、リスクに見合ったリターンが得られるのかを判断する指標
値が大きい方がいい
Calmar Ratio	カルマーレシオ
年間利益率と最大損失率の比で計算される
値が大きい方がいい
Max. Drawdown [%]	最大下落率
Avg. Drawdown [%]	平均下落率
Max. Drawdown Duration	最大下落期間
Max. Drawdown Duration	平均下落期間
Trades	取引回数
Win Rate [%]	勝率
Best Trade [%]	1回の取引における最大利益
Worst Trade [%]	1回の取引における最大損失
Avg. Trade [%]	1回の取引のおける平均損益
Max. Trade Duration	最大取引期間
Avg. Trade Duration	平均取引期間
Profit Factor	利益と損失の比,プラスなら利益が期待できる
Expectancy [%]	期待値
SQN	SystemQualityNumberの略らしい。。。
取引システムの分類（評価）方法の1つ目安の数字まとめました。とりあえず３くらいを目指せばいい。
# 1.6-1.9：平均以下
# 2.0-2.4：平均
# 2.5-2.9：良い
# 3.0-5.0：素晴らしい
# 5.1-6.9：最高
# 7.0～：神レベル
_strategy	使用したStrategyの関数名
_equity_curve	資産推移データ
_trades	各取引データ
"""

'\nStart\tバックテスト開始日時\nEnd\tバックテスト終了日時\nDuration\tバックテスト期間\nExposure Time [%]\tポジションを保有していた期間の割合\nEquity Final [$]\t最終的な資金\nEquity Peak [$]\t資金の最大値\nReturn [%]\t損益\nBuy & Hold Retun [%]\tバイ＆ホールドしていた場合の損益\nReturn (Ann.) [%]\t年間損益\nVolatility (Ann.） [%]\t年間損益のボラティリティ\nSharp Ratio\tシャープレシオ\nリターンとリスク（標準偏差）の比から計算される\n値が大きい方がいい\nSortino Ratio\tソルティノレシオ\n下落時のリスクを考慮して、リスクに見合ったリターンが得られるのかを判断する指標\n値が大きい方がいい\nCalmar Ratio\tカルマーレシオ\n年間利益率と最大損失率の比で計算される\n値が大きい方がいい\nMax. Drawdown [%]\t最大下落率\nAvg. Drawdown [%]\t平均下落率\nMax. Drawdown Duration\t最大下落期間\nMax. Drawdown Duration\t平均下落期間\nTrades\t取引回数\nWin Rate [%]\t勝率\nBest Trade [%]\t1回の取引における最大利益\nWorst Trade [%]\t1回の取引における最大損失\nAvg. Trade [%]\t1回の取引のおける平均損益\nMax. Trade Duration\t最大取引期間\nAvg. Trade Duration\t平均取引期間\nProfit Factor\t利益と損失の比,プラスなら利益が期待できる\nExpectancy [%]\t期待値\nSQN\tSystemQualityNumberの略らしい。。。\n取引システムの分類（評価）方法の1つ目安の数字まとめました。とりあえず３くらいを目指せばいい。\n# 1.6-1.9：平均以下\n# 2.0-2.4：平均\n# 2.5-2.9：良い\n# 3.0-5.0：素晴らしい\n# 5.1-6.9：最高\n# 7.0～：神レベル\n_strategy\t使用し

gio: file:///home/shunta/py_venvs/py1/BackTest/SmaCross.html: No application is registered as handling this file


In [10]:
#SMAの最適化
output2=bt.optimize(
    n1=range(10, 70, 5), 
    n2=range(10, 70, 5), 
    maximize='Equity Final [$]', 
    constraint=lambda 
    p: p.n1 < p.n2
)

print(output2)
bt.plot()

Start                     2022-01-03 00:00:00
End                       2023-11-28 00:00:00
Duration                    694 days 00:00:00
Exposure Time [%]                    40.91858
Equity Final [$]                 12079.479743
Equity Peak [$]                  12987.836296
Return [%]                          20.794797
Buy & Hold Return [%]                4.609637
Return (Ann.) [%]                    10.44988
Volatility (Ann.) [%]               16.131017
Sharpe Ratio                         0.647813
Sortino Ratio                        1.106405
Calmar Ratio                         0.643145
Max. Drawdown [%]                  -16.248091
Avg. Drawdown [%]                   -2.434621
Max. Drawdown Duration      261 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      24.368316
Worst Trade [%]                     -4.690474
Avg. Trade [%]                    

Row(id='2511', ...)

gio: file:///home/shunta/py_venvs/py1/BackTest/SmaCross(n1=10,n2=65).html: No application is registered as handling this file
